In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv('2-fft-normal-n-0-3-data.csv')

In [3]:
df1.shape

(700549, 18)

In [19]:
df1 = df1.loc[df1['router'] == 1]

In [20]:
df1 = df1.drop(columns=['router'])

In [21]:
df1.to_csv('2-fft-normal-n-0-3-data-r1-good-single.csv',index=False)

In [22]:
df1 = pd.read_csv('2-fft-normal-n-0-3-data-r1-good-single.csv')

In [23]:
def timecount(df):
    timearr = []
    interval = 99
    count = 0
    for index, row in df.iterrows():
        if row["timestamp"]<=interval:
            count+=1
        else:
            timearr.append([interval+1,count])
            count=1
            interval+=100
    timearr.append([interval+1,count])
    return timearr

In [24]:
def maxcount(timearr,df):
    countarr = []
    increarr = []
    maxarr = []
    for i in range(len(timearr)):
        for cnt in range(timearr[i][1],0,-1):
            countarr.append(cnt)
            maxarr.append(timearr[i][1])
            increment = timearr[i][1] - cnt + 1
            increarr.append(increment)
    df = df.assign(packet_count_decr=countarr)
    df = df.assign(packet_count_incr=increarr)
    df = df.assign(max_packet_count=maxarr)
    return df

In [25]:
df1 = maxcount(timecount(df1),df1)

In [26]:
df1

,timestamp,outport,inport,cache_coherence_type,flit_id,flit_type,vnet,vc,src_ni,src_router,dst_ni,dst_router,traversal_id,hop_count,current_hop,hop_percentage,enqueue_time,packet_count_decr,packet_count_incr,max_packet_count
0,11,2,4,1,1,4,3,9,0,0,23,7,0,4,1,0.250000,8,2,1,2
1,84,4,2,1,1,4,3,9,3,3,24,8,2,5,2,0.400000,12,1,2,2
2,216,2,4,1,1,4,3,9,0,0,22,6,5,3,1,0.333333,8,1,1,1
3,342,2,4,1,1,4,3,9,0,0,23,7,7,4,1,0.250000,8,1,1,1
4,388,1,2,1,1,4,3,9,3,3,21,5,8,3,2,0.666667,12,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56912,3268816,4,2,6,2,2,3,9,18,2,20,4,20041,3,1,0.333333,15,5,1,5
56913,3268823,4,2,6,3,2,3,9,18,2,20,4,20041,3,1,0.333333,22,4,2,5
56914,3268830,4,2,6,4,2,3,9,18,2,20,4,20041,3,1,0.333333,29,3,3,5
56915,3268837,4,2,6,5,3,3,9,18,2,20,4,20041,3,1,0.333333,36,2,4,5


In [27]:
def rename(df):
    df['traversal_id'] = df['traversal_id']+1
    df["packet_count_index"] = df["packet_count_decr"]*df["packet_count_incr"]
    df["packet_max_index"] = df["packet_count_index"]*df["max_packet_count"]
    df["port_index"] = df["outport"]*df["inport"]
    df["cache_coherence_flit_index"] = df["cache_coherence_type"]*df["flit_id"]
    df["flit_index"] = df["cache_coherence_flit_index"]*df["flit_type"]
    df["traversal_index"] = df["flit_index"]*df["traversal_id"]
    df["cache_coherence_vnet_index"] = df["cache_coherence_type"]*df["vnet"]
    df["vnet_vc_index"] = df["vnet"]*df["vc"]
    df["vnet_vc_cc_index"] = df["vnet"]*df["cache_coherence_vnet_index"]
    df.head(50)

In [28]:
rename(df1)

In [29]:
df1['target'] = 1

In [30]:
df1

,timestamp,outport,inport,cache_coherence_type,flit_id,flit_type,vnet,vc,src_ni,src_router,...,packet_count_index,packet_max_index,port_index,cache_coherence_flit_index,flit_index,traversal_index,cache_coherence_vnet_index,vnet_vc_index,vnet_vc_cc_index,target
0,11,2,4,1,1,4,3,9,0,0,...,2,4,8,1,4,4,3,27,9,1
1,84,4,2,1,1,4,3,9,3,3,...,2,4,8,1,4,12,3,27,9,1
2,216,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,24,3,27,9,1
3,342,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,32,3,27,9,1
4,388,1,2,1,1,4,3,9,3,3,...,1,1,2,1,4,36,3,27,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56912,3268816,4,2,6,2,2,3,9,18,2,...,5,25,8,12,24,481008,18,27,54,1
56913,3268823,4,2,6,3,2,3,9,18,2,...,8,40,8,18,36,721512,18,27,54,1
56914,3268830,4,2,6,4,2,3,9,18,2,...,9,45,8,24,48,962016,18,27,54,1
56915,3268837,4,2,6,5,3,3,9,18,2,...,8,40,8,30,90,1803780,18,27,54,1


In [31]:
df1.dtypes

timestamp                       int64
outport                         int64
inport                          int64
cache_coherence_type            int64
flit_id                         int64
flit_type                       int64
vnet                            int64
vc                              int64
src_ni                          int64
src_router                      int64
dst_ni                          int64
dst_router                      int64
traversal_id                    int64
hop_count                       int64
current_hop                     int64
hop_percentage                float64
enqueue_time                    int64
packet_count_decr               int64
packet_count_incr               int64
max_packet_count                int64
packet_count_index              int64
packet_max_index                int64
port_index                      int64
cache_coherence_flit_index      int64
flit_index                      int64
traversal_index                 int64
cache_cohere

In [32]:
df1.to_csv('2-fft-normal-n-0-3-data-r1-good-single.csv',index=False)